# Conclusions

Our team handled the task of analyzing improvement in time of computation when applying parallelization to the code on Natural Language Processing models. Our main work was performed on a web-scraped dataset provided by the [National Cybersecurity Center](https://www.ncsc.gov.uk/), which we documented [here](https://github.com/Galeforse/DST-Assessment-05/blob/main/Data/NCSC%20Reports.csv). The results of our test indicated a notable speed-up in performance, which we ultimately considered a moderate success. However, there were plenty of instances that require more in-depth analysis regarding our method. We will formally address a few of them here, before finishing the report.

Our first means of implementing parallelization was done in the web-scraping stage itself, where we achieved (see [Matt's folder](https://github.com/Galeforse/DST-Assessment-05/tree/main/Matt%20Corrie)) impressive amounts of speed-up by MultiProcess, trivial parallelization. The results achieved are as follows:

In [8]:
values = pickle.load(open('../Data/Web Scraping Times.p','rb'))
times = pickle.load(open('../Data/Web Scraping Times.p','rb'))

times = ['No Parallelisation', 'Parallelisation', 'Asynchronous Parallelisation']

d = {'Implementation': times, 'Times': values}
df = pd.DataFrame(data=d)
df

,Implementation,Times
0,No Parallelisation,17:40.70
1,Parallelisation,06:49.86
2,Asynchronous Parallelisation,07:12.74


Upon obtaining the dataset, we deployed a TF-IDF BoW (Bag of Words) model on the articles section. These was done in 3 ways: First a non-parallelized run, using a single worker; Second, a trivially parallelized run using MultiCore processing with 4 workers and lastly, the core of our work - parallelization using PySpark's pipelines method(see [Alex's folder](https://github.com/Galeforse/DST-Assessment-05/blob/main/Alex%20Caian/Assessment%205.ipynb)). The results of this process were as follows:

In [2]:
import pickle
import pandas as pd

t1 = pickle.load(open('../Data/Time_No_Parallelization.p', 'rb'))
t2 = pickle.load(open('../Data/Time_MultiCore_Parallelization.p', 'rb'))
t3 = pickle.load(open('../Data/Time_PySpark_Parallelization.p', 'rb'))

times = ['No Parallelisation', 'Multi Core Parallelisation', 'Pyspark Parallelisation']

d = {'Implementation': times, 'Times': [t1,t2,t3]}
df = pd.DataFrame(data=d)
df

,Implementation,Times
0,No Parallelisation,00:10:13.528705
1,Multi Core Parallelisation,00:09:47.233116
2,Pyspark Parallelisation,00:03:20.515291


Notably, we achieved a runtime of only 32.87% of the trivially parallelized variant and 31.28% of the non-parallelized variant within the final and desired implementation. The times were documented, extracted, saved and loaded using the 'pickle' library.

Lastly, our team focused on alternative implementations of our inference goal via Spark, which do _not_ use the NLP dedicated extension of Spark. These attempts, while running accordingly for the most part, with the exception of more difficulty at the implementation stage, provided no added value to the results previously achieved by SparkNLP - which easily outperformed other Spark methods for the purpose of topic modelling we chose(see [Gabriel's folder](https://github.com/Galeforse/DST-Assessment-05/tree/main/Gabriel%20Grant)).

We therefore concluded that the best approach towards achieving temporal optimization via means of parallelization in the context of topic modelling is via SparkNLP. 

## Further Reflections

**What would have improved the speedup?**

A main factor determining how impressive the speedup would be is certainly the model we chose to apply it on. Our choice was an NLP, more specifically, the Term Frequency - Inverse Document Frequency method which requires a broad (recommended over 10000 words) dictionary to run efficiently. In light of this point of view, our data was lacking from the start. Given the nature of its purpose, the vocabulary was a limiting factor all throughout. Therefore, we're inclined to believe that a richer dictionary would have overall improved the speedup. Evidently, this statement refers to an improvement in the "Parallelized v.s not-Parallelized" proportion, **not** in the actual runtime - as a larger dataset is trivially adding to the computational complexity. However, we claim that it wouldn't be a linear addition to the time complexity when computing in parallel, since every kth datapoint would be distributed via the pipelines, resulting in an n/k increase in time complexity for an n increase in size complexity. Apart from potential code optimization, another way the speedup could've been improved was by removing the topic which proved massive dissimilarity from the other within our visualizations. Notably in [section 2.2 of this part of our work](https://github.com/Galeforse/DST-Assessment-05/blob/main/Alex%20Caian/Assessment%205.ipynb), within the visualizations, one can easily notice the segregated topic. This may prove difficult to deal with in the context of parallel computation, perhaps less so than when each topic was individually processed sequentially. Comparing the "Parallelized v.s not-Parallelized" variants of our models posterior to discarding that topic might have surprising aftermaths, worth taking into account.

**Why was there such a small difference between 'Non-Parallelized' and 'Trivially Parallelized'?**

In truth, several seeds (runs) of the same code were attempted on various numbers of multiple workers, ranging from 2 to 8, to test performance against the non-parallelized version. These tests have not made it into the final paper as they otherwise posed no real importance towards our inference goal - that is - setting up and using PySpark/SparkNLP. However, they can easily be replicated within a 'for loop' to test the results, with an expected runtime of ~ 85 minutes. The results of these tests were surprising at the very least. Setting up the amount of workers to any value other than 3 or 4 actually resulted in a _time slowdown_ rather than speedup when compared to the sequential implementation. At first, our team was baffled by these absurdly counterintuitive results. We chose to display within the final report the version with 4 workers, which resulted in a time reduction of less than 30 seconds compared to the non-parallelized code, or 95.7% of the original code. The better question is hence 'Why did the non-parallelized code usually outperform the parallelized one?'.

This was briefly touched on in [section 2.2](https://github.com/Galeforse/DST-Assessment-05/blob/main/Alex%20Caian/Assessment%205.ipynb). We isolated the reason as mainly having to do with the way MultiCore processing handles the NLP task. Instead of computing the TF-IDF in parallel by Articles, perhaps the algorithm is computing the TF-IDFs of each Article in parallel. Moreover, since the IDF is always longer to compute compared to the TF it has to wait for itself to finish before moving in to the next article - which, in fact, may take longer than the speedup it offers via parallelization. Therefore, the overall time would oddly take more than were we to compute it sequentially. However, we must stress that this remains a hypothesis. One clear way to test it, however, would be by increasing the size of our dataset. This can easily be achieved by duplicating the data any number of times desired. If after duplication the difference (Non_parallelized_time - Trivially_parallelized_time) decreases proportionally to the increase in data size, but also better than linearly proportional, then it is clear that our hypothesis is likely true - since by increasing the data size we ensure that parallelization becomes worth implementing. In our case, the data size is rather small and can thus be computed sequentially with little to no drawbacks.

## Contributors
 _Link to the contributor's Git profile ~ Link to their work on this project_

[Matt Corrie](https://github.com/mc17336) ~ [Work #1](https://github.com/Galeforse/DST-Assessment-04/tree/main/Matt%20Corrie)

[Alex Caian](https://github.com/Alex-Caian) ~ [Work #2](https://github.com/Galeforse/DST-Assessment-04/tree/main/Alex%20Caian)
    
[Gabriel Grant](https://github.com/Galeforse) ~ [Work #3](https://github.com/Galeforse/DST-Assessment-04/tree/main/Gabriel%20Grant)